<a href="https://colab.research.google.com/github/Anjasfedo/eceg-lsb-lzw-huffman/blob/main/ECEG/eceg_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECEG

In [ ]:
import random

class Point:
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y

    def is_infinity(self):
        """Check if the point is the point at infinity."""
        return self.x is None and self.y is None

    def __eq__(self, other):
        """Custom equality check for Point objects."""
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False

    def __hash__(self):
        """Make Point hashable by defining a unique hash."""
        return hash((self.x, self.y))

    def __repr__(self):
        if self.is_infinity():
            return "Point at Infinity"
        return f"Point({self.x}, {self.y})"

class ECEG:
  def __init__(self):
    self.a = 6
    self.b = 7
    self.p = 61
    self.base_point = self.generate_random_valid_point()

    self.characters = [
        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
        'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
        'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
        'y', 'z', '0', '1', '2', '3', '4', '5',
        '6', '7', '8', '9', ',', '.', '/', '?',
        ':', ';', '[', ']', '{', '}', '\\', '|',
        '‘', ' ', '!', '@', '#', '$', '%', '^',
        '&', '*', '(', ')', '-', '_', '=', '+'
    ][:63]  # Trim the list to 63 characters


    self.valid_points = self.get_all_points()
    self.point_to_char, self.char_to_point = self.create_mappings()

  def elliptic_curve_equation(self, x):
    return (x**3 + self.a*x + self.b) % self.p

  # def is_on_curve(self, x, y):
  #   return self.elliptic_curve_equation(x)  == (y**2) % self.p

  # def generate_random_point(self):
  #   while True:
  #     x = random.randint(1, self.p - 1)
  #     y = random.randint(1, self.p - 1)
  #     if self.is_on_curve(x, y):
  #       return Point(x, y)

  def is_on_curve(self, x, y):
        """Check if a point (x, y) lies on the curve."""
        if x is None or y is None:
            return False
        return (y**2 - (x**3 + self.a * x + self.b)) % self.p == 0

  def generate_random_valid_point(self):
        """Generate a random point that lies on the elliptic curve."""
        while True:
            x = random.randint(0, self.p - 1)  # Random x-coordinate
            y_squared = (x**3 + self.a * x + self.b) % self.p  # Compute y^2

            # Check if y_squared is a quadratic residue modulo p
            if pow(y_squared, (self.p - 1) // 2, self.p) == 1:
                # Find a valid y-coordinate
                for y in range(self.p):
                    if (y**2) % self.p == y_squared:
                        return Point(x, y)

  # def calc_point_add(self, P, Q):
  #   R = Point()  # Initialize the result point R
  #   print(P, Q)

  #   slope = ((Q.y - P.y) / (Q.x - P.x))

  #   # Calculate Rx
  #   R.x = (slope**2 - P.x - Q.x)

  #   # Calculate Ry
  #   R.y = (slope * (P.x - R.x) - P.y)

  #   return R

  def calc_point_add(self, P, Q):
    """Calculate the addition of two points P and Q on the elliptic curve."""
    R = Point()  # Initialize the result point R

    if P.is_infinity():
        return Q
    if Q.is_infinity():
        return P

    # Handle the case where P and Q are inverses
    if P.x == Q.x and (P.y != Q.y or P.y == 0):
        return Point()  # Point at infinity

    # Calculate slope
    if P.x == Q.x and P.y == Q.y:
        # Point doubling
        slope = (3 * P.x**2 + self.a) * pow(2 * P.y, -1, self.p) % self.p
    else:
        # Regular point addition
        slope = (Q.y - P.y) * pow(Q.x - P.x, -1, self.p) % self.p

    # Calculate Rx
    R.x = (slope**2 - P.x - Q.x) % self.p

    # Calculate Ry
    R.y = (slope * (P.x - R.x) - P.y) % self.p

    return R

  def calc_point_doubling(self, P):
      """Calculate the point doubling 2P = P + P on the elliptic curve."""
      R = Point()  # Initialize the result point R

      if P.is_infinity() or P.y == 0:
          # Point at infinity for vertical tangent or zero y-coordinate
          return Point()

      # Calculate slope for point doubling
      slope = (3 * P.x**2 + self.a) * pow(2 * P.y, -1, self.p) % self.p

      # Calculate Rx
      R.x = (slope**2 - 2 * P.x) % self.p

      # Calculate Ry
      R.y = (slope * (P.x - R.x) - P.y) % self.p

      return R

  # def calc_point_subtraction(self, P, Q):
  #     """Calculate the subtraction of two points P - Q on the elliptic curve."""
  #     # Negate Q to get -Q
  #     Q_neg = Point(Q.x, (-Q.y) % self.p)

  #     # Add P and -Q
  #     return self.calc_point_add(P, Q_neg)

  def calc_point_subtraction(self, P, Q):
    """Calculate the subtraction of two points P - Q on the elliptic curve."""
    # Check if Q is the point at infinity
    if Q.is_infinity():
        return P  # P - Q = P when Q is the point at infinity

    # Check if P is the point at infinity
    if P.is_infinity():
        Q_neg = Point(Q.x, (-Q.y) % self.p)  # Negate Q
        return Q_neg  # 0 - Q = -Q

    # Negate Q to get -Q
    Q_neg = Point(Q.x, (-Q.y) % self.p)

    # Add P and -Q
    return self.calc_point_add(P, Q_neg)


  def calc_point_multiplication(self, P, k):
    """Calculate kP using the double-and-add method."""
    R = Point()  # Start with the point at infinity
    current_point = P

    while k > 0:
        if k % 2 == 1:
            # If the current bit is 1, add the current point
            R = self.calc_point_add(R, current_point)
        # Double the point
        current_point = self.calc_point_add(current_point, current_point)
        k //= 2  # Move to the next bit

    return R

  def generate_keys(self):
        """Generate a private and public key pair."""
        # Private key d: Random scalar
        private_key = random.randint(1, self.p - 1)

        # Public key e2 = d * e1 (base point)
        public_key = self.calc_point_multiplication(self.base_point, private_key)

        return private_key, public_key

  def encrypt(self, plaintext_point, public_key, k=None):
      """
      Encrypt a point on the elliptic curve using the public key.

      Args:
          plaintext_point (Point): The plaintext point to encrypt.
          public_key (Point): The public key.
          k (int, optional): The ephemeral key for deterministic testing. If None, a random k is used.

      Returns:
          tuple: A tuple containing the cipher points (C1, C2).
      """
      # Generate a random ephemeral key k if not provided
      if k is None:
          k = random.randint(1, self.p - 1)

      # Calculate C1 = k * e1 (base point)
      C1 = self.calc_point_multiplication(self.base_point, k)

      # Calculate k * e2 (public key)
      k_e2 = self.calc_point_multiplication(public_key, k)

      # Calculate C2 = P + k * e2
      C2 = self.calc_point_add(plaintext_point, k_e2)

      return C1, C2


  def decrypt(self, C1, C2, private_key):
        """
        Decrypt a ciphertext pair (C1, C2) using the private key.

        Args:
            C1 (Point): The first ciphertext point.
            C2 (Point): The second ciphertext point.
            private_key (int): The private key (d).

        Returns:
            Point: The plaintext point (P).
        """
        # Calculate d * C1
        d_C1 = self.calc_point_multiplication(C1, private_key)

        # Subtract d * C1 from C2 to get the plaintext point
        plaintext_point = self.calc_point_subtraction(C2, d_C1)

        return plaintext_point

  # def get_all_points(self):
  #       """
  #       Generate all valid points on the elliptic curve.

  #       Returns:
  #           list: A list of all valid points on the elliptic curve, including the point at infinity.
  #       """
  #       points = [Point()]  # Start with the point at infinity
  #       for x in range(self.p):
  #           y_squared = (x**3 + self.a * x + self.b) % self.p
  #           for y in range(self.p):
  #               if (y**2) % self.p == y_squared:
  #                   points.append(Point(x, y))
  #       return points

  def get_all_points(self):
      """
      Generate all valid points on the elliptic curve.

      Returns:
          list: A list of all valid points on the elliptic curve, including the point at infinity.
      """
      points = [Point()]  # Start with the point at infinity
      for x in range(self.p):
          y_squared = self.elliptic_curve_equation(x)
          for y in range(self.p):
              if (y**2) % self.p == y_squared:
                  points.append(Point(x, y))
      return points



  # def koblitz_encode(self, m, k=20):
  #     """
  #     Encode a number m as a point on the elliptic curve using Koblitz encoding.

  #     Args:
  #         m (int): The number to encode.
  #         k (int): The multiplier for encoding.

  #     Returns:
  #         Point: The encoded point on the elliptic curve.
  #     """
  #     if m <= 0:
  #         raise ValueError(f"Number m must be positive. Received: {m}")

  #     # Iterate to find a valid x value
  #     for i in range(1, k + 1):
  #         x = (k * m + i) % self.p  # Ensure x is within the field range
  #         y_squared = self.elliptic_curve_equation(x)
  #         if pow(y_squared, (self.p - 1) // 2, self.p) == 1:  # Check if y^2 is a quadratic residue
  #             # Find the corresponding y value
  #             for y in range(self.p):
  #                 if (y**2) % self.p == y_squared:
  #                     return Point(x, y)

  #     raise ValueError(f"Failed to encode number {m} as a valid point.")


  # def koblitz_decode(self, point, k=20):
  #     """
  #     Decode a point on the elliptic curve back to a number m.

  #     Args:
  #         point (Point): The point to decode.
  #         k (int): The multiplier used in encoding.

  #     Returns:
  #         int: The decoded number m.
  #     """
  #     if point.is_infinity():
  #         raise ValueError("Cannot decode the point at infinity.")

  #     x = point.x

  #     # Iterate to find the correct m value
  #     for m in range(1, self.p):
  #         if (k * m + 1) % self.p == x or (k * m + 2) % self.p == x:
  #             return m

  #     raise ValueError(f"Point {point} does not decode to a valid number.")

  # def is_quadratic_residue(self, a, mod):
  #   return pow(a, (mod - 1) // 2, mod) == 1

  # def koblitz_encode(self, m, k=100, used_points=None):
  #     """
  #     Encode a number m as a valid point on the elliptic curve, ensuring no duplicates.

  #     Args:
  #         m (int): The number to encode.
  #         k (int): The multiplier for encoding.
  #         used_points (set): A set of already-used points to avoid duplicates.

  #     Returns:
  #         Point: The encoded point on the elliptic curve.
  #     """
  #     if used_points is None:
  #         used_points = set()

  #     if m < 1 or m >= self.p:
  #         raise ValueError(f"Number m must be in the range [1, {self.p - 1}]. Received: {m}")

  #     for n in range(1, self.p):  # Iterate through offsets
  #         x = (k * m + n) % self.p
  #         y_squared = self.elliptic_curve_equation(x)

  #         # Explicitly check for y^2 = 0 (y = 0)
  #         if y_squared == 0:
  #             point = Point(x, 0)
  #             if point not in used_points:
  #                 used_points.add(point)
  #                 return point

  #         # Otherwise, proceed with quadratic residue check
  #         if self.is_quadratic_residue(y_squared, self.p):
  #             for y in range(self.p):
  #                 if (y**2) % self.p == y_squared:
  #                     point = Point(x, y)
  #                     if point not in used_points:
  #                         used_points.add(point)
  #                         return point

  #     raise ValueError(f"Failed to encode number {m} as a valid point after {self.p} attempts.")

  def create_mappings(self):
    valid_points = [point for point in self.valid_points if not point.is_infinity()]
    # print(f"Valid Points: {len(valid_points)}, Characters: {len(self.characters)}")
    if len(valid_points) != len(self.characters):
        raise ValueError("Mismatch between the number of valid points and characters.")
    point_to_char = {point: char for point, char in zip(valid_points, self.characters)}
    char_to_point = {char: point for point, char in point_to_char.items()}
    return point_to_char, char_to_point


  def encode_character(self, char):
        """Encode a character to a point on the elliptic curve."""
        if char not in self.char_to_point:
            raise ValueError(f"Character '{char}' not in mapping.")
        return self.char_to_point[char]

  def decode_point(self, point):
        """Decode a point on the elliptic curve to a character."""
        if point not in self.point_to_char:
            raise ValueError(f"Point '{point}' not in mapping.")
        return self.point_to_char[point]

  # def koblitz_decode(self, point, k=20):
  #   n = 1
  #   while True:
  #     m = (point.x - n) // k
  #     if m * k + n == point.x:
  #       # print(point.x)
  #       # print(point.x - n + 1)
  #       return m
  #     n += 1

  # def encrypt_message(self, message, public_key):
  #     """
  #     Encrypt a message using the elliptic curve encryption scheme.

  #     Args:
  #         message (str): The message to encrypt.
  #         public_key (Point): The public key to use for encryption.

  #     Returns:
  #         list: A list of encrypted tuples (C1, C2) representing the ciphertext.
  #     """
  #     ciphertext = []

  #     for char in message:
  #         # Encode character to a point
  #         plaintext_point = self.encode_character(char)

  #         # Encrypt the point
  #         C1, C2 = self.encrypt(plaintext_point, public_key)

  #         # Append the encrypted tuple to the ciphertext
  #         ciphertext.append((C1, C2))

  #         print(f"Character '{char}' encoded to {plaintext_point}, encrypted as (C1: {C1}, C2: {C2})")

  #     return ciphertext

  def encrypt_message(self, message, public_key):
      """
      Encrypt a message using the elliptic curve encryption scheme and return a character-based ciphertext.

      Args:
          message (str): The message to encrypt.
          public_key (Point): The public key to use for encryption.

      Returns:
          str: The encrypted message as a string of characters.
      """
      ciphertext = ""

      for char in message:
          # Encode character to a point
          plaintext_point = self.encode_character(char)

          # Encrypt the point
          C1, C2 = self.encrypt(plaintext_point, public_key)

          # Decode encrypted points back to characters
          encrypted_char_C1 = self.decode_point(C1)
          encrypted_char_C2 = self.decode_point(C2)

          # Append the encrypted characters to the ciphertext
          ciphertext += encrypted_char_C1 + encrypted_char_C2

          # print(f"Character '{char}' encoded to {plaintext_point}, encrypted as (C1: {C1}, C2: {C2}), "
          #       f"and represented as '{encrypted_char_C1}{encrypted_char_C2}'.")
      print(ciphertext)
      return ciphertext

  def decrypt_message(self, ciphertext, private_key):
      """
      Decrypt a ciphertext into its plaintext message using the private key.

      Args:
          ciphertext (str): The encrypted message as a string of characters.
          private_key (int): The private key for decryption.

      Returns:
          str: The decrypted plaintext message.
      """
      plaintext = ""
      # Process ciphertext two characters at a time (C1 and C2)
      for i in range(0, len(ciphertext), 2):
          # Decode C1 and C2 back into points
          C1 = self.encode_character(ciphertext[i])
          C2 = self.encode_character(ciphertext[i + 1])

          # Decrypt the point
          decrypted_point = self.decrypt(C1, C2, private_key)

          # Decode the point back into the original character
          char = self.decode_point(decrypted_point)
          plaintext += char

          # print(f"Ciphertext pair '{ciphertext[i]}{ciphertext[i + 1]}' decrypted to point {decrypted_point}, "
          #       f"decoded to character '{char}'.")

      return plaintext


# Test

In [ ]:
import unittest

class TestECEG(unittest.TestCase):
    def setUp(self):
        # Initialize the elliptic curve
        self.elliptic = ECEG()

        # Generate keys
        self.private_key, self.public_key = self.elliptic.generate_keys()

        # Generate a plaintext point for encryption
        self.plaintext_point = self.elliptic.generate_random_valid_point()

    def test_point_on_curve(self):
        """Test if a randomly generated point lies on the elliptic curve."""
        self.assertTrue(
            self.elliptic.is_on_curve(self.elliptic.base_point.x, self.elliptic.base_point.y),
            f"Point is not on the curve. Point: {self.elliptic.base_point}"
        )
        print(f"Point on curve test passed. Point: {self.elliptic.base_point}")

    def test_addition_doubling(self):
        """Test if P + P equals 2P."""
        result_add = self.elliptic.calc_point_add(self.elliptic.base_point, self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_add.x, result_add.y),
            f"Addition result is not on the curve. Result: {result_add}"
        )

        result_doubling = self.elliptic.calc_point_doubling(self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_doubling.x, result_doubling.y),
            f"Doubling result is not on the curve. Result: {result_doubling}"
        )

        self.assertEqual(
            result_add,
            result_doubling,
            f"P + P does not equal 2P. Addition Result: {result_add}, Doubling Result: {result_doubling}"
        )
        print(f"Addition and doubling test passed. P + P = 2P. Result: {result_add}")

    def test_calc_addition_subtraction(self):
        """Test if (P + P) - P equals P."""
        result_add = self.elliptic.calc_point_add(self.elliptic.base_point, self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_add.x, result_add.y),
            f"Addition result is not on the curve. Result: {result_add}"
        )

        result_subtraction = self.elliptic.calc_point_subtraction(result_add, self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_subtraction.x, result_subtraction.y),
            f"Subtraction result is not on the curve. Result: {result_subtraction}"
        )

        self.assertEqual(
            result_subtraction,
            self.elliptic.base_point,
            f"(P + P) - P does not equal P. Subtraction Result: {result_subtraction}, Random Point: {self.elliptic.base_point}"
        )
        print(f"Addition and subtraction test passed. (P + P) - P = P. Result: {result_subtraction}")

    def test_calc_doubling_multiplication(self):
        """Test if 2P doubling 2P equals 4P."""
        result_doubling_first = self.elliptic.calc_point_doubling(self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_doubling_first.x, result_doubling_first.y),
            f"Doubling result first is not on the curve. Result: {result_doubling_first}"
        )

        result_doubling_second = self.elliptic.calc_point_doubling(result_doubling_first)
        self.assertTrue(
            self.elliptic.is_on_curve(result_doubling_second.x, result_doubling_second.y),
            f"Doubling result second is not on the curve. Result: {result_doubling_second}"
        )

        k = 4
        result_multiplication = self.elliptic.calc_point_multiplication(self.elliptic.base_point, k)
        self.assertTrue(
            self.elliptic.is_on_curve(result_multiplication.x, result_multiplication.y),
            f"Multiplication result is not on the curve. Result: {result_multiplication}"
        )

        self.assertEqual(
            result_doubling_second,
            result_multiplication,
            f"2P doubling 2P does not equal 4P. Doubling Result: {result_doubling_second}, Multiplication Result: {result_multiplication}"
        )
        print(f"Doubling and multiplication test passed. 2P doubling 2P = 4P. Result: {result_multiplication}")

    def test_generate_keys(self):
        """Test if the key generation produces a valid private-public key pair."""
        private_key, public_key = self.elliptic.generate_keys()
        self.assertGreater(
            private_key, 0,
            f"Private key must be greater than 0. Private Key: {private_key}"
        )
        self.assertLess(
            private_key, self.elliptic.p,
            f"Private key must be less than {self.elliptic.p}. Private Key: {private_key}"
        )

        self.assertTrue(
            self.elliptic.is_on_curve(public_key.x, public_key.y),
            f"Public key does not lie on the curve. Public Key: {public_key} (private_key={private_key}, base_point={self.elliptic.base_point})"
        )

        expected_public_key = self.elliptic.calc_point_multiplication(self.elliptic.base_point, private_key)
        self.assertEqual(
            public_key,
            expected_public_key,
            f"Public key is not correctly calculated as d * e1. Expected: {expected_public_key}, Actual: {public_key}"
        )
        print(f"Key generation test passed. Private Key: {private_key}, Public Key: {public_key}")

    def test_encrypt(self):
        """Test if the encryption function generates valid cipher points."""
        C1, C2 = self.elliptic.encrypt(self.plaintext_point, self.public_key)
        self.assertTrue(
            self.elliptic.is_on_curve(C1.x, C1.y),
            f"C1 does not lie on the curve. C1: {C1}"
        )
        self.assertTrue(
            self.elliptic.is_on_curve(C2.x, C2.y),
            f"C2 does not lie on the curve. C2: {C2}"
        )
        print(f"Encryption test passed. Cipher Points: C1={C1}, C2={C2}")

    def test_decrypt(self):
        """Test if decryption recovers the original plaintext."""
        C1, C2 = self.elliptic.encrypt(self.plaintext_point, self.public_key)
        decrypted_point = self.elliptic.decrypt(C1, C2, self.private_key)
        self.assertEqual(
            decrypted_point,
            self.plaintext_point,
            f"Decryption failed. Decrypted Point: {decrypted_point}, Original Point: {self.plaintext_point}"
        )
        print(f"Decryption test passed. Decrypted Point: {decrypted_point}, Original Point: {self.plaintext_point} with cipher {C1, C2}")

    def test_get_all_points(self):
        """Test if the method returns all 64 valid points on the elliptic curve."""
        # Generate all points on the curve
        all_points = self.elliptic.get_all_points()

        # Check the total number of points
        expected_total_points = 64 # 64 for a=6, b=7, p=61
        self.assertEqual(
            len(all_points), expected_total_points,
            f"Expected {expected_total_points} points, but got {len(all_points)}."
        )

        # Verify that all points lie on the curve
        for point in all_points:
            if point.is_infinity():
                # Skip the point at infinity
                continue
            self.assertTrue(
                self.elliptic.is_on_curve(point.x, point.y),
                f"Point {point} does not lie on the curve."
            )

        print(f"All {len(all_points)} points are valid and lie on the curve.")

    def test_character_list(self):
        """Test if the characters list matches the number of valid points and is unique."""
        # Adjusted expected characters to match the valid points
        expected_characters = [
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
            'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
            'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
            'y', 'z', '0', '1', '2', '3', '4', '5',
            '6', '7', '8', '9', ',', '.', '/', '?',
            ':', ';', '[', ']', '{', '}', '\\', '|',
            '‘', ' ', '!', '@', '#', '$', '%', '^',
            '&', '*', '(', ')', '-', '_', '=', '+'
        ][:len(self.elliptic.characters)]  # Trim to match the actual number of characters

        # Check the total number of characters
        self.assertEqual(
            len(self.elliptic.characters), len(self.elliptic.valid_points) - 1,
            f"Characters list must contain {len(self.elliptic.valid_points) - 1} elements, "
            f"but got {len(self.elliptic.characters)}."
        )

        # Verify the characters match the expected list
        self.assertListEqual(
            self.elliptic.characters, expected_characters,
            "Characters list does not match the expected set of characters."
        )

        # Check for duplicates in the list
        unique_characters = set(self.elliptic.characters)
        self.assertEqual(
            len(unique_characters), len(self.elliptic.characters),
            f"Characters list contains duplicates. Unique count: {len(unique_characters)}."
        )

        # Check if the space character is included (if applicable)
        if ' ' in expected_characters:
            self.assertIn(
                ' ', self.elliptic.characters,
                "Characters list does not include the space character."
            )

        print("Character list test passed. All characters are unique and valid.")


    def test_curve_parameters(self):
        """Test if the curve parameters a, b, and p are correctly set."""
        # Expected values for a, b, and p
        expected_a = 6
        expected_b = 7
        expected_p = 61

        # Check if a is correct
        self.assertEqual(
            self.elliptic.a, expected_a,
            f"Parameter a is incorrect. Expected: {expected_a}, Found: {self.elliptic.a}"
        )

        # Check if b is correct
        self.assertEqual(
            self.elliptic.b, expected_b,
            f"Parameter b is incorrect. Expected: {expected_b}, Found: {self.elliptic.b}"
        )

        # Check if p is correct
        self.assertEqual(
            self.elliptic.p, expected_p,
            f"Parameter p is incorrect. Expected: {expected_p}, Found: {self.elliptic.p}"
        )

        print(f"Curve parameters test passed. a={self.elliptic.a}, b={self.elliptic.b}, p={self.elliptic.p}")


    # def test_koblitz_encode_decode(self):
    #     """
    #     Test Koblitz encoding and decoding for a range of numbers and compare to all valid points.
    #     """
    #     k = 20
    #     used_points = set()
    #     encoded_points = []

    #     for m in range(1, 65):
    #         try:
    #             encoded_point = self.elliptic.koblitz_encode(m, k, used_points)
    #             self.assertTrue(
    #                 self.elliptic.is_on_curve(encoded_point.x, encoded_point.y),
    #                 f"Encoded point {encoded_point} for number {m} is not on the curve."
    #             )
    #             encoded_points.append(encoded_point)
    #         except ValueError as e:
    #             print(f"Encoding failed for m={m}: {e}")
    #             raise

    #     all_valid_points = self.elliptic.get_all_points()

    #     # Check each encoded point
    #     for idx, encoded_point in enumerate(encoded_points, start=1):
    #         try:
    #             self.assertIn(
    #                 encoded_point, all_valid_points,
    #                 f"Encoded point {encoded_point} at index {idx} is not in the list of valid points."
    #             )
    #         except AssertionError as e:
    #             print(f"Error with point {encoded_point}: not found in valid points.")
    #             raise

    #     print(f"Encoded points ({len(encoded_points)}) match valid points on the curve.")

    # def test_koblitz_vs_all_valid(self):
    #     """
    #     Compare Koblitz-encoded points with all valid points to identify missing points.
    #     """
    #     k = 100
    #     used_points = set()
    #     encoded_points = []

    #     # Encode numbers 1 to 64
    #     for m in range(1, 65):
    #         try:
    #             encoded_point = self.elliptic.koblitz_encode(m, k, used_points)
    #             encoded_points.append(encoded_point)
    #         except ValueError:
    #             print(f"Failed to encode number {m}.")
    #             continue

    #     # Get all valid points, excluding the point at infinity
    #     all_valid_points = [point for point in self.elliptic.get_all_points() if not point.is_infinity()]

    #     # Check for missing points
    #     missing_points = [point for point in all_valid_points if point not in encoded_points]

    #     print(f"Encoded points: {len(encoded_points)}")
    #     print(f"All valid points: {len(all_valid_points)}")
    #     print(f"Missing points: {missing_points}")

    #     self.assertEqual(
    #         len(encoded_points), len(all_valid_points),
    #         f"Expected {len(all_valid_points)} encoded points, but got {len(encoded_points)}."
    #     )
    #     self.assertFalse(
    #         missing_points,
    #         f"Missing points: {missing_points}."
    #     )

    def test_mapping_validity(self):
        """Test if the number of valid points matches the number of characters."""
        valid_points = [point for point in self.elliptic.valid_points if not point.is_infinity()]
        self.assertEqual(
            len(valid_points), len(self.elliptic.characters),
            f"Mismatch: {len(valid_points)} valid points vs {len(self.elliptic.characters)} characters."
        )
        print(f"Mapping validity test passed. Valid points: {len(valid_points)}, Characters: {len(self.elliptic.characters)}")

    def test_unique_mappings(self):
        """Test if each point maps to a unique character and vice versa."""
        # Check point-to-character mapping
        mapped_characters = set(self.elliptic.point_to_char.values())
        self.assertEqual(
            len(mapped_characters), len(self.elliptic.characters),
            "Point-to-character mapping contains duplicates."
        )
        print(f"Point-to-character mapping is unique. Total mapped characters: {len(mapped_characters)}")

        # Check character-to-point mapping
        mapped_points = set(self.elliptic.char_to_point.values())
        self.assertEqual(
            len(mapped_points), len(self.elliptic.valid_points) - 1,  # Excluding the point at infinity
            "Character-to-point mapping contains duplicates."
        )
        print(f"Character-to-point mapping is unique. Total mapped points: {len(mapped_points)}")

    def test_encode_character(self):
        """Test if encoding a character returns the correct point."""
        for char in self.elliptic.characters:
            point = self.elliptic.encode_character(char)
            self.assertTrue(
                self.elliptic.is_on_curve(point.x, point.y),
                f"Encoded point {point} for character '{char}' is not on the curve."
            )
            # print(f"Character '{char}' successfully encoded to point {point}.")
        print("Character encoding test passed.")

    def test_decode_point(self):
        """Test if decoding a point returns the correct character."""
        for char, point in self.elliptic.char_to_point.items():
            decoded_char = self.elliptic.decode_point(point)
            self.assertEqual(
                decoded_char, char,
                f"Decoded character '{decoded_char}' does not match original character '{char}'."
            )
            # print(f"Point {point} successfully decoded to character '{char}'.")
        print("Point decoding test passed.")

    def test_round_trip(self):
        """Test if encoding and then decoding a character returns the original character."""
        for char in self.elliptic.characters:
            point = self.elliptic.encode_character(char)
            decoded_char = self.elliptic.decode_point(point)
            self.assertEqual(
                char, decoded_char,
                f"Round-trip failed: Character '{char}' encoded to {point} and decoded to '{decoded_char}'."
            )
            # print(f"Round-trip successful: Character '{char}' -> Point {point} -> Character '{decoded_char}'.")
        print("Round-trip test passed.")

    def test_invalid_character_encoding(self):
        """Test if encoding an invalid character raises an error."""
        invalid_char = '?@'
        with self.assertRaises(ValueError):
            self.elliptic.encode_character(invalid_char)
        print(f"Invalid character encoding test passed. Attempted to encode invalid character: '{invalid_char}'")

    def test_invalid_point_decoding(self):
        """Test if decoding an invalid point raises an error."""
        invalid_point = Point(999, 999)  # Point not on the curve
        with self.assertRaises(ValueError):
            self.elliptic.decode_point(invalid_point)
        print(f"Invalid point decoding test passed. Attempted to decode invalid point: {invalid_point}")

    def test_encrypt_message(self):
        """Test if a message is correctly encrypted into a ciphertext and represented as characters."""
        # Generate keys
        private_key, public_key = self.elliptic.generate_keys()

        # Message to encrypt
        message = "hello"

        # Fixed ephemeral key for testing
        fixed_k = 5

        # Encrypt the message
        ciphertext = self.elliptic.encrypt_message(message, public_key)

        # Validate ciphertext length
        self.assertEqual(
            len(ciphertext), len(message) * 2,
            f"Ciphertext length {len(ciphertext)} does not match expected {len(message) * 2}."
        )

        # Validate each character mapping
        # for i, char in enumerate(message):
        #     # Encode character to point
        #     plaintext_point = self.elliptic.encode_character(char)

        #     # Encrypt point manually with fixed k for testing
        #     C1, C2 = self.elliptic.encrypt(plaintext_point, public_key, k=fixed_k)

        #     # Decode C1 and C2 back to characters
        #     encrypted_char_C1 = self.elliptic.decode_point(C1)
        #     encrypted_char_C2 = self.elliptic.decode_point(C2)

        #     # Assert the ciphertext matches the manually generated encrypted characters
        #     self.assertEqual(
        #         ciphertext[2 * i], encrypted_char_C1,
        #         f"Ciphertext character at position {2 * i} does not match: Expected {encrypted_char_C1}, Found {ciphertext[2 * i]}"
        #     )
        #     self.assertEqual(
        #         ciphertext[2 * i + 1], encrypted_char_C2,
        #         f"Ciphertext character at position {2 * i + 1} does not match: Expected {encrypted_char_C2}, Found {ciphertext[2 * i + 1]}"
        #     )

        #     print(f"Character '{char}' encrypted to (C1: {C1}, C2: {C2}), represented as '{encrypted_char_C1}{encrypted_char_C2}'.")

        print("Message encryption test passed.")

    def test_decrypt_message(self):
        """Test if a ciphertext is correctly decrypted back into the original plaintext message."""
        # Generate keys
        private_key, public_key = self.elliptic.generate_keys()

        # Original message to encrypt
        original_message = "hello"

        # Encrypt the message
        ciphertext = self.elliptic.encrypt_message(original_message, public_key)

        # Decrypt the message
        decrypted_message = self.elliptic.decrypt_message(ciphertext, private_key)

        # Validate the decrypted message
        self.assertEqual(
            decrypted_message, original_message,
            f"Decrypted message '{decrypted_message}' does not match original message '{original_message}'."
        )

        print(f"Original message '{original_message}' encrypted to ciphertext '{ciphertext}', "
              f"successfully decrypted back to '{decrypted_message}'.")



# Run the unittest tests
if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)


test_addition_doubling (__main__.TestECEG)
Test if P + P equals 2P. ... ok
test_calc_addition_subtraction (__main__.TestECEG)
Test if (P + P) - P equals P. ... ok
test_calc_doubling_multiplication (__main__.TestECEG)
Test if 2P doubling 2P equals 4P. ... ok
test_character_list (__main__.TestECEG)
Test if the characters list matches the number of valid points and is unique. ... ok
test_curve_parameters (__main__.TestECEG)
Test if the curve parameters a, b, and p are correctly set. ... ok
test_decode_point (__main__.TestECEG)
Test if decoding a point returns the correct character. ... ok
test_decrypt (__main__.TestECEG)
Test if decryption recovers the original plaintext. ... ok
test_decrypt_message (__main__.TestECEG)
Test if a ciphertext is correctly decrypted back into the original plaintext message. ... ok
test_encode_character (__main__.TestECEG)
Test if encoding a character returns the correct point. ... ok
test_encrypt (__main__.TestECEG)
Test if the encryption function generates v

Addition and doubling test passed. P + P = 2P. Result: Point(26, 54)
Addition and subtraction test passed. (P + P) - P = P. Result: Point(47, 30)
Doubling and multiplication test passed. 2P doubling 2P = 4P. Result: Point(57, 23)
Character list test passed. All characters are unique and valid.
Curve parameters test passed. a=6, b=7, p=61
Point decoding test passed.
Decryption test passed. Decrypted Point: Point(32, 24), Original Point: Point(32, 24) with cipher (Point(53, 39), Point(46, 18))
xeeo=l.uxf
Original message 'hello' encrypted to ciphertext 'xeeo=l.uxf', successfully decrypted back to 'hello'.
Character encoding test passed.
Encryption test passed. Cipher Points: C1=Point(54, 7), C2=Point(3, 33)
ex=\p xty\
Message encryption test passed.
Key generation test passed. Private Key: 47, Public Key: Point(17, 52)
All 64 points are valid and lie on the curve.
Invalid character encoding test passed. Attempted to encode invalid character: '?@'
Invalid point decoding test passed. Attem

In [ ]:
# Step 1: Initialize the ECEG class
elliptic = ECEG()

# Step 2: Generate private and public keys
private_key, public_key = elliptic.generate_keys()
print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")

# Step 3: Define a message to encrypt
original_message = "hello"
print(f"Original Message: {original_message}")

# Step 4: Encrypt the message
ciphertext = elliptic.encrypt_message(original_message, public_key)
print(f"Encrypted Ciphertext: {ciphertext}")

# Step 5: Decrypt the ciphertext back into the original message
decrypted_message = elliptic.decrypt_message(ciphertext, private_key)
print(f"Decrypted Message: {decrypted_message}")

# Step 6: Validate the result
assert decrypted_message == original_message, "Decryption failed: Messages do not match!"
print("Encryption and decryption successful!")


Private Key: 3
Public Key: Point(46, 43)
Original Message: hello
{@=\=i(u=g
Encrypted Ciphertext: {@=\=i(u=g
Decrypted Message: hello
Encryption and decryption successful!


In [ ]:
len(ciphertext)

10

In [ ]:
# Step 3: Define a message to encrypt
original_message = "hello"
print(f"Original Message: {original_message}")

# Step 4: Encrypt the message
ciphertext = elliptic.encrypt_message(original_message, public_key)
print(f"Encrypted Ciphertext: {ciphertext}")

# Step 5: Decrypt the ciphertext back into the original message
decrypted_message = elliptic.decrypt_message(ciphertext, private_key)
print(f"Decrypted Message: {decrypted_message}")

# Step 6: Validate the result
assert decrypted_message == original_message, "Decryption failed: Messages do not match!"
print("Encryption and decryption successful!")

Original Message: hello
}n=\}]=i&m
Encrypted Ciphertext: }n=\}]=i&m
Decrypted Message: hello
Encryption and decryption successful!


In [ ]:
    # self.point_to_char, self.char_to_point = self.create_mappings()

elliptic.point_to_char

{Point(1, 21): 'a',
 Point(1, 40): 'b',
 Point(2, 24): 'c',
 Point(2, 37): 'd',
 Point(3, 28): 'e',
 Point(3, 33): 'f',
 Point(4, 20): 'g',
 Point(4, 41): 'h',
 Point(6, 25): 'i',
 Point(6, 36): 'j',
 Point(9, 27): 'k',
 Point(9, 34): 'l',
 Point(11, 1): 'm',
 Point(11, 60): 'n',
 Point(13, 5): 'o',
 Point(13, 56): 'p',
 Point(15, 19): 'q',
 Point(15, 42): 'r',
 Point(17, 9): 's',
 Point(17, 52): 't',
 Point(20, 21): 'u',
 Point(20, 40): 'v',
 Point(21, 0): 'w',
 Point(26, 7): 'x',
 Point(26, 54): 'y',
 Point(27, 24): 'z',
 Point(27, 37): '0',
 Point(28, 17): '1',
 Point(28, 44): '2',
 Point(29, 29): '3',
 Point(29, 32): '4',
 Point(30, 15): '5',
 Point(30, 46): '6',
 Point(32, 24): '7',
 Point(32, 37): '8',
 Point(34, 29): '9',
 Point(34, 32): ',',
 Point(40, 21): '.',
 Point(40, 40): '/',
 Point(41, 0): '?',
 Point(42, 7): ':',
 Point(42, 54): ';',
 Point(43, 17): '[',
 Point(43, 44): ']',
 Point(46, 18): '{',
 Point(46, 43): '}',
 Point(47, 30): '\\',
 Point(47, 31): '|',
 Point(50,

In [ ]:
elliptic.char_to_point

{'a': Point(1, 21),
 'b': Point(1, 40),
 'c': Point(2, 24),
 'd': Point(2, 37),
 'e': Point(3, 28),
 'f': Point(3, 33),
 'g': Point(4, 20),
 'h': Point(4, 41),
 'i': Point(6, 25),
 'j': Point(6, 36),
 'k': Point(9, 27),
 'l': Point(9, 34),
 'm': Point(11, 1),
 'n': Point(11, 60),
 'o': Point(13, 5),
 'p': Point(13, 56),
 'q': Point(15, 19),
 'r': Point(15, 42),
 's': Point(17, 9),
 't': Point(17, 52),
 'u': Point(20, 21),
 'v': Point(20, 40),
 'w': Point(21, 0),
 'x': Point(26, 7),
 'y': Point(26, 54),
 'z': Point(27, 24),
 '0': Point(27, 37),
 '1': Point(28, 17),
 '2': Point(28, 44),
 '3': Point(29, 29),
 '4': Point(29, 32),
 '5': Point(30, 15),
 '6': Point(30, 46),
 '7': Point(32, 24),
 '8': Point(32, 37),
 '9': Point(34, 29),
 ',': Point(34, 32),
 '.': Point(40, 21),
 '/': Point(40, 40),
 '?': Point(41, 0),
 ':': Point(42, 7),
 ';': Point(42, 54),
 '[': Point(43, 17),
 ']': Point(43, 44),
 '{': Point(46, 18),
 '}': Point(46, 43),
 '\\': Point(47, 30),
 '|': Point(47, 31),
 '‘': Poin

# Dump code

In [ ]:
"""
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-27-7c20c24e4453> in <cell line: 6>()
      4
      5 # Step 4: Encrypt the message
----> 6 ciphertext = elliptic.encrypt_message(original_message, public_key)
      7 print(f"Encrypted Ciphertext: {ciphertext}")
      8

1 frames
<ipython-input-19-35a0f4bd0633> in decode_point(self, point)
    384         """Decode a point on the elliptic curve to a character."""
    385         if point not in self.point_to_char:
--> 386             raise ValueError(f"Point '{point}' not in mapping.")
    387         return self.point_to_char[point]
    388

ValueError: Point 'Point at Infinity' not in mapping.
"""

In [ ]:
"""
error list:

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-155-0a96d71a6351>", line 112, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=16, base_point=Point(53, 22))
-

-
FAIL: test_calc_doubling_multiplication (__main__.TestECEG)
Test if 2P doubling 2P equals 4P.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-170-0a96d71a6351>", line 76, in test_calc_doubling_multiplication
    self.assertTrue(
AssertionError: False is not true : Doubling result second is not on the curve. Result: Point at Infinity (result_doubling_first=Point(21, 0)
-

-
FAIL: test_calc_doubling_multiplication (__main__.TestECEG)
Test if 2P doubling 2P equals 4P.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-175-0a96d71a6351>", line 76, in test_calc_doubling_multiplication
    self.assertTrue(
AssertionError: False is not true : Doubling result second is not on the curve. Result: Point at Infinity (result_doubling_first=Point(41, 0) )
-

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-175-0a96d71a6351>", line 112, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=56, base_point=Point(54, 7))
-

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-180-0a96d71a6351>", line 112, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=56, base_point=Point(32, 24))
-

-
FAIL: test_calc_doubling_multiplication (__main__.TestECEG)
Test if 2P doubling 2P equals 4P.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-187-0a96d71a6351>", line 76, in test_calc_doubling_multiplication
    self.assertTrue(
AssertionError: False is not true : Doubling result second is not on the curve. Result: Point at Infinity (result_doubling_first=Point(60, 0) )
-

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-196-0a96d71a6351>", line 112, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=48, base_point=Point(50, 14))
-

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-211-802f94a9058a>", line 115, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=16, base_point=Point(17, 9))
-

-
ERROR: test_decrypt (__main__.TestECEG)
Test if decryption recovers the original plaintext.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-221-16ec17c776f0>", line 151, in test_decrypt
    decrypted_point = self.elliptic.decrypt(C1, C2, self.private_key)
  File "<ipython-input-216-cd268fb89798>", line 198, in decrypt
    plaintext_point = self.calc_point_subtraction(C2, d_C1)
  File "<ipython-input-216-cd268fb89798>", line 127, in calc_point_subtraction
    Q_neg = Point(Q.x, (-Q.y) % self.p)
TypeError: bad operand type for unary -: 'NoneType'
-



"""

'\nerror list:\n\n-\nFAIL: test_generate_keys (__main__.TestECEG)\nTest if the key generation produces a valid private-public key pair.\n----------------------------------------------------------------------\nTraceback (most recent call last):\n  File "<ipython-input-155-0a96d71a6351>", line 112, in test_generate_keys\n    self.assertTrue(\nAssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=16, base_point=Point(53, 22))\n-\n\n-\nFAIL: test_calc_doubling_multiplication (__main__.TestECEG)\nTest if 2P doubling 2P equals 4P.\n----------------------------------------------------------------------\nTraceback (most recent call last):\n  File "<ipython-input-170-0a96d71a6351>", line 76, in test_calc_doubling_multiplication\n    self.assertTrue(\nAssertionError: False is not true : Doubling result second is not on the curve. Result: Point at Infinity (result_doubling_first=Point(21, 0)\n-\n\n-\nFAIL: test_calc_doubling_multiplic

## test case

In [ ]:
def test_calc_add_and_doubling():
    elliptic = ECEG()

    # Generate a valid random point
    random_point = elliptic.generate_random_point()
    print(f"Random Point: {random_point}")

    # Ensure the generated point lies on the curve
    assert elliptic.is_on_curve(random_point.x, random_point.y), "Generated point is not on the curve."

    # Perform point addition (P + P)
    result_add = elliptic.calc_point_add(random_point, random_point)
    print(f"Result of P + P (Addition): {result_add}")

    # Perform point doubling (2P)
    result_doubling = elliptic.calc_point_doubling(random_point)
    print(f"Result of 2P (Doubling): {result_doubling}")

    # Ensure both addition and doubling results lie on the curve
    assert elliptic.is_on_curve(result_add.x, result_add.y), "Addition result is not on the curve."
    assert elliptic.is_on_curve(result_doubling.x, result_doubling.y), "Doubling result is not on the curve."

    if (result_add == result_doubling):
      print("Test passed: Point addition and doubling produce the same result.")
    else:
      Exception("Test failed: Point addition and doubling do not produce the same result.")

# Run the test
test_calc_add_and_doubling()

AttributeError: 'ECEG' object has no attribute 'generate_random_point'

In [ ]:
def test_calc_subtraction():
  elliptic = ECEG()
  random_point = elliptic.generate_random_point()
  print(random_point)

  result_add = elliptic.calc_point_add(random_point, random_point)
  print(result_add)

  result_subtraction = elliptic.calc_point_subtraction(result_add, random_point)
  print(result_subtraction)

  assert result_subtraction != random_point, "should be same"

test_calc_subtraction()

In [ ]:
def test_calc_multiplication():
  elliptic = ECEG()
  random_point = elliptic.generate_random_point()
  print(random_point)

  result_doubling = elliptic.calc_point_doubling(random_point)
  print(result_doubling)

  result_doubling_2 = elliptic.calc_point_doubling(result_doubling)
  print(result_doubling_2)

  result_multiplication = elliptic.calc_point_multiplication(random_point, 2)
  print(result_multiplication)

  assert result_multiplication != result_doubling_2, "should be same"

test_calc_multiplication()

In [ ]:
5 * 2 * 2, 5 * 2